In [1]:
import pandas as pd 

In [2]:
df = pd.read_json('MongoDB/airlines_reviews.json',lines=True)
df

,_id,Title,Name,ReviewDate,Airline,Verified,Reviews,TypeofTraveller,MonthFlown,Route,Class,SeatComfort,StaffService,FoodnBeverages,InflightEntertainment,ValueForMoney,OverallRating,Recommended
0,66b85f58c4a8d6813c87d8d1,Flight was amazing,Alison Soetantyo,1/3/2024,Singapore Airlines,TRUE,Flight was amazing. The crew onboard this fl...,Solo Leisure,Dec-23,Jakarta to Singapore,Business Class,4,4,4,4,4,9,yes
1,66b85f58c4a8d6813c87d8d2,seats on this aircraft are dreadful,Robert Watson,21/2/2024,Singapore Airlines,TRUE,Booking an emergency exit seat still meant hug...,Solo Leisure,Feb-24,Phuket to Singapore,Economy Class,5,3,4,4,1,3,no
2,66b85f58c4a8d6813c87d8d3,Food was plentiful and tasty,S Han,20/2/2024,Singapore Airlines,TRUE,Excellent performance on all fronts. I would...,Family Leisure,Feb-24,Siem Reap to Singapore,Economy Class,1,5,2,1,5,10,yes
3,66b85f58c4a8d6813c87d8d4,how much food was available,D Laynes,19/2/2024,Singapore Airlines,TRUE,Pretty comfortable flight considering I was f...,Solo Leisure,Feb-24,Singapore to London Heathrow,Economy Class,5,5,5,5,5,10,yes
4,66b85f58c4a8d6813c87d8d5,service was consistently good,A Othman,19/2/2024,Singapore Airlines,TRUE,The service was consistently good from start ...,Family Leisure,Feb-24,Singapore to Phnom Penh,Economy Class,5,5,5,5,5,10,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1574,66b85f58c4a8d6813c87def7,They have best customer service,Laxmi Thapa,26/6/2020,Qatar Airways,TRUE,We were flying round trip to KTM. We got stu...,Couple Leisure,Jun-20,Kathmandu to Dallas via Doha,Economy Class,5,5,4,1,5,10,yes
1575,66b85f58c4a8d6813c87def8,engaging and hospitable as ever,Brett Stewart,16/6/2020,Qatar Airways,TRUE,I was hesitant about taking the long way hom...,Business,Jun-20,Chicago to Sydney via Doha,Business Class,5,5,5,5,5,10,yes
1576,66b85f58c4a8d6813c87def9,told I could claim a refund,H meares,12/6/2020,Qatar Airways,TRUE,London to Auckland via Doha on 31st May. My ...,Solo Leisure,Sep-19,London to Auckland via Doha,Economy Class,5,1,5,5,3,1,no
1577,66b85f58c4a8d6813c87defa,crew members are so nice,Jasvinder Gill,20/5/2020,Qatar Airways,TRUE,Amritsar to Vancouver via Doha. Best airline...,Couple Leisure,May-20,Amritsar to Vancouver via Doha,Economy Class,4,5,4,5,1,8,yes


In [3]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

def preprocess_text(text):
    if pd.isnull(text):
        return ""
    text = re.sub(r"http\S+", "", text)  # Remove URLs
    text = re.sub(r"[^a-zA-Z\s]", "", text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    text = text.strip()  # Remove leading and trailing whitespaces
    return text


# Apply preprocessing
df['cleaned_title'] = df['Title'].apply(preprocess_text)
df['cleaned_review'] = df['Reviews'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tjuir\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tjuir\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tjuir\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
from textblob import TextBlob

# Function to analyze sentiment for each review
def analyze_review_sentiment(review):
    if pd.isnull(review):
        return {"polarity": None, "subjectivity": None}  # Handle missing reviews
    text_blob = TextBlob(review)
    return {
        "polarity": text_blob.sentiment.polarity,
        "subjectivity": text_blob.sentiment.subjectivity
    }

# Apply the function to each review and create new columns for sentiment scores
df[['polarity', 'subjectivity']] = df['cleaned_title'].apply(lambda x: pd.Series(analyze_review_sentiment(x)))
df[['polarity_review', 'subjectivity_review']] = df['cleaned_review'].apply(lambda x: pd.Series(analyze_review_sentiment(x)))
# Display the updated DataFrame with sentiment analysis
print(df[['Reviews', 'polarity', 'subjectivity']].head())


                                             Reviews  polarity  subjectivity
0    Flight was amazing. The crew onboard this fl...       0.6           0.9
1  Booking an emergency exit seat still meant hug...      -1.0           1.0
2    Excellent performance on all fronts. I would...       0.0           0.0
3   Pretty comfortable flight considering I was f...       0.3           0.3
4   The service was consistently good from start ...       0.7           0.6


In [5]:
df['complaints'] = df['polarity'].apply(lambda x: 'yes' if x < 0 else 'no')
df['polarity'].head(10)

0    0.600000
1   -1.000000
2    0.000000
3    0.300000
4    0.700000
5   -1.000000
6   -0.325000
7    0.000000
8   -0.375000
9   -0.071429
Name: polarity, dtype: float64

In [6]:
df = df.drop(['subjectivity','polarity_review', 'subjectivity_review','cleaned_review'],axis = 1)
df

,_id,Title,Name,ReviewDate,Airline,Verified,Reviews,TypeofTraveller,MonthFlown,Route,...,SeatComfort,StaffService,FoodnBeverages,InflightEntertainment,ValueForMoney,OverallRating,Recommended,cleaned_title,polarity,complaints
0,66b85f58c4a8d6813c87d8d1,Flight was amazing,Alison Soetantyo,1/3/2024,Singapore Airlines,TRUE,Flight was amazing. The crew onboard this fl...,Solo Leisure,Dec-23,Jakarta to Singapore,...,4,4,4,4,4,9,yes,flight was amazing,0.6,no
1,66b85f58c4a8d6813c87d8d2,seats on this aircraft are dreadful,Robert Watson,21/2/2024,Singapore Airlines,TRUE,Booking an emergency exit seat still meant hug...,Solo Leisure,Feb-24,Phuket to Singapore,...,5,3,4,4,1,3,no,seats on this aircraft are dreadful,-1.0,yes
2,66b85f58c4a8d6813c87d8d3,Food was plentiful and tasty,S Han,20/2/2024,Singapore Airlines,TRUE,Excellent performance on all fronts. I would...,Family Leisure,Feb-24,Siem Reap to Singapore,...,1,5,2,1,5,10,yes,food was plentiful and tasty,0.0,no
3,66b85f58c4a8d6813c87d8d4,how much food was available,D Laynes,19/2/2024,Singapore Airlines,TRUE,Pretty comfortable flight considering I was f...,Solo Leisure,Feb-24,Singapore to London Heathrow,...,5,5,5,5,5,10,yes,how much food was available,0.3,no
4,66b85f58c4a8d6813c87d8d5,service was consistently good,A Othman,19/2/2024,Singapore Airlines,TRUE,The service was consistently good from start ...,Family Leisure,Feb-24,Singapore to Phnom Penh,...,5,5,5,5,5,10,yes,service was consistently good,0.7,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1574,66b85f58c4a8d6813c87def7,They have best customer service,Laxmi Thapa,26/6/2020,Qatar Airways,TRUE,We were flying round trip to KTM. We got stu...,Couple Leisure,Jun-20,Kathmandu to Dallas via Doha,...,5,5,4,1,5,10,yes,they have best customer service,1.0,no
1575,66b85f58c4a8d6813c87def8,engaging and hospitable as ever,Brett Stewart,16/6/2020,Qatar Airways,TRUE,I was hesitant about taking the long way hom...,Business,Jun-20,Chicago to Sydney via Doha,...,5,5,5,5,5,10,yes,engaging and hospitable as ever,0.4,no
1576,66b85f58c4a8d6813c87def9,told I could claim a refund,H meares,12/6/2020,Qatar Airways,TRUE,London to Auckland via Doha on 31st May. My ...,Solo Leisure,Sep-19,London to Auckland via Doha,...,5,1,5,5,3,1,no,told i could claim a refund,0.0,no
1577,66b85f58c4a8d6813c87defa,crew members are so nice,Jasvinder Gill,20/5/2020,Qatar Airways,TRUE,Amritsar to Vancouver via Doha. Best airline...,Couple Leisure,May-20,Amritsar to Vancouver via Doha,...,4,5,4,5,1,8,yes,crew members are so nice,0.6,no


In [7]:
df.to_csv('airlines_reviews_clean.csv')